## HW4
### Brandon Shurick


>### HW 4.0.   
What is MrJob? How is it different to Hadoop MapReduce?   
What are the mapper_final(), combiner_final(), reducer_final() methods? When are they called?

>### HW 4.1  
What is serialization in the context of MrJob or Hadoop?   
When it used in these frameworks?   
What is the default serialization mode for input and outputs for MrJob?   

>### HW 4.2  
Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:

>https://kdd.ics.uci.edu/databases/msweb/msweb.html  
>http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

>This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.  

>Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:  

>C,"10001",10001   #Visitor id 10001  
V,1000,1          #Visit by Visitor 10001 to page id 1000  
V,1001,1          #Visit by Visitor 10001 to page id 1001  
V,1002,1          #Visit by Visitor 10001 to page id 1002  
C,"10002",10002   #Visitor id 10001  
V
Note: #denotes comments  
to the format:  

>V,1000,1,C, 10001  
V,1001,1,C, 10001  
V,1002,1,C, 10001  

>Write the python code to accomplish this.  

In [91]:
%%writefile parse_logs.py
#!/usr/bin/env python
import sys
customer = None
urls = {}

## Process line-by-line ##
for line in sys.stdin:
    line = line.strip()
    fields = line.split(',')
    
    ## Add URL to dictionary ## 
    if fields[0]=='A':
        idnum = fields[1]
        url = fields[4]
        urls[idnum] = url
    
    ## Get customer ID ## 
    if fields[0]=='C':
        customer = fields[2]
        
    ## Print each V line ##
    elif fields[0]=='V':
        print(','.join((line,'C',customer)))

## Write url dictionary to a file ##
with open('urls_dict.py','w') as w:
    print >>w, urls


Overwriting parse_logs.py


In [92]:
!chmod +x parse_logs.py

In [95]:
!cat ./anonymous-msweb.data | ./parse_logs.py > msweb_formatted.txt

In [96]:
!head msweb_formatted.txt

V,1000,1,C,10001
V,1001,1,C,10001
V,1002,1,C,10001
V,1001,1,C,10002
V,1003,1,C,10002
V,1001,1,C,10003
V,1003,1,C,10003
V,1004,1,C,10003
V,1005,1,C,10004
V,1006,1,C,10005


>### HW 4.3:  
Find the 5 most frequently visited pages using mrjob from the output of 4.2 (i.e., transfromed log file).

In [97]:
%%writefile page_frequency.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep

class FrequentPages(MRJob):
    def mapper_pages(self, _, line):
        ''' split CSV lines and emit page ID '''
        fields = line.split(',')
        if fields[0]=='V':
            yield fields[1], int(fields[2])
    
    def combiner_pages(self, page, cnts):
        ''' sum counts by page id '''
        totals = sum(cnts)
        yield page, totals
    
    def reducer_pages(self, page, cnts):
        ''' sum counts by page id '''
        totals = sum(cnts)
        yield None, (totals, page)
    
    def reducer_top_pages(self, _ , totalspages):
        ''' Sort pages and emit top 5'''
        sortedpages = sorted(totalspages
                             ,key=lambda x: x[0]
                             , reverse=True)
        for i in range(5):
            yield i+1,sortedpages[i]
    
    def steps(self):
        return [MRStep(mapper=self.mapper_pages,
                        reducer=self.reducer_pages),
               MRStep(reducer=self.reducer_top_pages)]

if __name__=='__main__':
    FrequentPages.run()

Overwriting page_frequency.py


In [98]:
!python page_frequency.py msweb_formatted.txt

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/page_frequency.bshur.20160204.203400.572972

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/page_frequency.bshur.20160204.203400.572972/step-0-mapper_part-00000
Counters from step 1:
  (no counters found)
writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/page_frequency.bshur.20160204.203400.572972/step-0-mapper-sorted
> sort /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/page_frequency.bshur.20160204.203400.572972/step-0-mapper_part-00000
writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/page_frequency.bshur.20160204.203400.572972/step-

>### HW 4.4:  
Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

In [99]:
%%writefile visitor_frequency.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep

class FrequentVisitors(MRJob):
    def init_mapper(self):
        ''' read in dictionary of URLs '''
        urls = {}
        with open('/Users/bshur/School/ML at Scale/HW4/urls_dict.py','r') as r:
            urls = eval(r.readlines()[0])
        self.urls = urls
    
    def mapper_pageusers(self, _, line):
        ''' split CSV lines and emit user and page ID '''
        fields = line.split(',')
        if fields[0]=='V':
            yield (self.urls[fields[1]], fields[1], fields[4]), int(fields[2])
    
    def combiner_pageusers(self, pageuser, cnts):
        ''' sum counts by page id '''
        totals = sum(cnts)
        yield pageuser, totals
    
    def reducer_pageusers(self, pageuser, cnts):
        ''' sum counts by page id '''
        totals = sum(cnts)
        yield pageuser, totals
    
    def mapper_page(self, pageuser, totals):
        ''' break pageuser into page and user '''
        page, pageid, user = pageuser
        yield (page, pageid), (user, totals)
    
    def reducer_page(self, pagepageid, usertotals):
        ''' choose top user for each page '''
        sortedtotals = sorted(usertotals, key=lambda x: x[1], reverse=True)
        maxuser = sortedtotals[0]
        yield pagepageid, maxuser[0]
    
    def steps(self):
        return [MRStep(mapper_init=self.init_mapper,
                        mapper=self.mapper_pageusers,
                        combiner=self.combiner_pageusers,
                        reducer=self.reducer_pageusers),
                MRStep(mapper=self.mapper_page,
                        reducer=self.reducer_page)]

if __name__=='__main__':
    FrequentVisitors.run()

Overwriting visitor_frequency.py


In [100]:
!python visitor_frequency.py ./msweb_formatted.txt

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/visitor_frequency.bshur.20160204.203403.487141

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/visitor_frequency.bshur.20160204.203403.487141/step-0-mapper_part-00000
Counters from step 1:
  (no counters found)
writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/visitor_frequency.bshur.20160204.203403.487141/step-0-mapper-sorted
> sort /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/visitor_frequency.bshur.20160204.203403.487141/step-0-mapper_part-00000
writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/visitor_frequency.bshur.20160204.2034

>### HW 4.5  
>Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:  

>0: Human, where only basic human-human communication is observed.  

>1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).  

>2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).  

>3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )  

>Check out the preprints of our recent research,  
>which spawned this dataset:  

>http://arxiv.org/abs/1505.04342  
>http://arxiv.org/abs/1508.01843  

>The main data lie in the accompanying file:  
topUsers_Apr-Jul_2014_1000-words.txt  
and are of the form:  

>USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...  
.
.

>where  

>USERID = unique user identifier  
CODE = 0/1/2/3 class code  
TOTAL = sum of the word counts  

>Using this data, you will implement a 1000-dimensional K-means algorithm on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.  

>Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:  

>(A) K=4 uniform random centroid-distributions over the 1000 words  
>(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution  
>(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution  
>(D) K=4 "trained" centroids, determined by the sums across the classes.  

>and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.  

>Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:  

>topUsers_Apr-Jul_2014_1000-words_summaries.txt

In [104]:
!head -n1 ./topUsers_Apr-Jul_2014_1000-words.txt

1180025371,2,1724608,75552,827,57603,7128,4282,45674,66811,27632,0,8,23783,2,42853,0,62335,22349,21428,19801,4125,0,0,2,1585,21118,1,1,1,16079,19676,1587,0,19695,0,0,0,0,0,0,2,20216,60,4278,0,16,46,788,2,0,0,3,0,3,0,0,111122,0,12,0,0,0,2,739,0,176,0,0,0,38,626,0,0,0,6,1584,0,19672,510,0,0,0,12,0,1675,0,0,0,0,5,2,0,0,1,9,0,0,31,0,0,2,0,0,0,0,4,64,476,0,1,0,617,0,0,15672,70315,70317,0,2997,0,0,0,665,0,0,12,0,0,0,3135,1,2,39,0,0,0,0,23,0,1,0,179,667,0,0,32,0,0,224,5,0,0,66,0,3,450,96,0,0,0,0,8,15,15,0,115,0,0,19672,0,46,15,0,0,2,0,51,0,0,0,298,0,0,5,2,165,3,0,0,46497,0,19675,0,4,0,42036,0,0,40035,84,0,103,0,2,12,1924,7,0,0,0,0,3,0,42629,197,15490,0,0,45,0,0,0,0,0,0,301,0,0,0,0,134,3300,0,422,386,0,19826,2,0,0,46,9,354,175,71,165,20338,0,109,0,1,44376,0,1370,0,0,0,0,0,0,0,0,0,0,2,0,0,4462,0,0,5,0,202,436,408,0,61,0,0,39888,74,0,19672,0,0,0,0,0,19672,19672,2,2,349,0,13,0,30,0,0,8,0,40,0,23,0,12,337,0,12,19952,26,0,0,15489,0,0,0,0,39,0,0,26,0,0,19,144,161,0,0,0,5558,0,23,1561,52,0,0,0,9,0,0,

In [139]:
%%writefile kmeans.py
#!/usr/bin/env python
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONProtocol, RawValueProtocol
import random 

class InitClusters(MRJob):
    INTERNAL_PROTOCOL = JSONProtocol
    OUTPUT_PROTOCOL = JSONProtocol
    
    def configure_options(self):
        super(InitClusters, self).configure_options()
        self.add_passthrough_option(
            '--k'
                , type='int'
                , dest='K'
                , default=4
                , help='Number of clusters')
    
    def map_data(self, _, line):
        fields = line.split(',')
        total = float(fields[2])
        point = [ float(i)/total for i in fields[3:] ]
        out = {}
        out['min'] = min(point)
        out['max'] = max(point)
        out['len'] = len(point)
        yield None, out
    
    def define_clusters(self, _, minmaxlen):
        minmaxlen = list(minmaxlen)
        maxval = max(m['max'] for m in minmaxlen)
        minval = min(m['min'] for m in minmaxlen)
        n_features = max(m['len'] for m in minmaxlen)
        k = self.options.K
        centroids = {}
        for i in range(k):
            centroids[i] = [ random.uniform(minval,maxval) 
                            for z in range(n_features) ]
        yield None, centroids
    
    def steps(self):
        return [MRStep(mapper=self.map_data
                       , reducer=self.define_clusters
                      )]
    
if __name__=='__main__':
    InitClusters.run()

Overwriting kmeans.py


In [140]:
!python kmeans.py topUsers_Apr-Jul_2014_1000-words.txt

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/kmeans.bshur.20160204.231520.959911

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/kmeans.bshur.20160204.231520.959911/step-0-mapper_part-00000
Counters from step 1:
  (no counters found)
writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/kmeans.bshur.20160204.231520.959911/step-0-mapper-sorted
> sort /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/kmeans.bshur.20160204.231520.959911/step-0-mapper_part-00000
writing to /var/folders/f8/70g84lyd3n387wjwhbfq4dp80000gn/T/kmeans.bshur.20160204.231520.959911/step-0-reducer_part-00000
Counters from step 